In [1]:
import os
import re
import pandas as pd

def BuildDataframe(directory):
    data = {'image_path': [], 'label': []}
    pattern = r'diseased'

    for plant in os.listdir(directory):
        class_dir = os.path.join(directory, plant)

        if os.path.isdir(class_dir):
            match = re.search(pattern, plant)
            label = "healthy"
            if match:
                label = "diseased"
                    
            print("Extracting ", plant, "...")            
            for image_file in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_file)
                    
                data['image_path'].append(image_path)
                data['label'].append(label)

    print("Data correctly extracted.")
    dataframe = pd.DataFrame(data)
    return dataframe

In [3]:
train_dir = r'C:\Users\user\Desktop\4.1 study\plant-leaves-for-image-classification\Plants_2\train'
train_dataframe = BuildDataframe(train_dir)

test_dir = r'C:\Users\user\Desktop\4.1 study\plant-leaves-for-image-classification\Plants_2\test'
test_dataframe = BuildDataframe(test_dir)

valid_dir = r'C:\Users\user\Desktop\4.1 study\plant-leaves-for-image-classification\Plants_2\valid'
valid_dataframe = BuildDataframe(valid_dir)

sample_df = train_dataframe.sample(n=10)
print(sample_df)

Extracting  Alstonia Scholaris diseased (P2a) ...
Extracting  Alstonia Scholaris healthy (P2b) ...
Extracting  Arjun diseased (P1a) ...
Extracting  Arjun healthy (P1b) ...
Extracting  Bael diseased (P4b) ...
Extracting  Basil healthy (P8) ...
Extracting  Chinar diseased (P11b) ...
Extracting  Chinar healthy (P11a) ...
Extracting  Gauva diseased (P3b) ...
Extracting  Gauva healthy (P3a) ...
Extracting  Jamun diseased (P5b) ...
Extracting  Jamun healthy (P5a) ...
Extracting  Jatropha diseased (P6b) ...
Extracting  Jatropha healthy (P6a) ...
Extracting  Lemon diseased (P10b) ...
Extracting  Lemon healthy (P10a) ...
Extracting  Mango diseased (P0b) ...
Extracting  Mango healthy (P0a) ...
Extracting  Pomegranate diseased (P9b) ...
Extracting  Pomegranate healthy (P9a) ...
Extracting  Pongamia Pinnata diseased (P7b) ...
Extracting  Pongamia Pinnata healthy (P7a) ...
Data correctly extracted.
Extracting  Alstonia Scholaris diseased (P2a) ...
Extracting  Alstonia Scholaris healthy (P2b) ...
Ex

In [4]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU config OK")
else:
    print("Error using GPU, CPU will be used")
tf.keras.backend.clear_session()


Error using GPU, CPU will be used



In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [6]:
input_shape = (224, 224, 3)
num_classes = 2
batch_size = 32
epochs = 10

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
print(train_dataframe)

                                             image_path     label
0     C:\Users\user\Desktop\4.1 study\plant-leaves-f...  diseased
1     C:\Users\user\Desktop\4.1 study\plant-leaves-f...  diseased
2     C:\Users\user\Desktop\4.1 study\plant-leaves-f...  diseased
3     C:\Users\user\Desktop\4.1 study\plant-leaves-f...  diseased
4     C:\Users\user\Desktop\4.1 study\plant-leaves-f...  diseased
...                                                 ...       ...
4269  C:\Users\user\Desktop\4.1 study\plant-leaves-f...   healthy
4270  C:\Users\user\Desktop\4.1 study\plant-leaves-f...   healthy
4271  C:\Users\user\Desktop\4.1 study\plant-leaves-f...   healthy
4272  C:\Users\user\Desktop\4.1 study\plant-leaves-f...   healthy
4273  C:\Users\user\Desktop\4.1 study\plant-leaves-f...   healthy

[4274 rows x 2 columns]


In [9]:
train_directory = r'C:\Users\user\Desktop\4.1 study\plant-leaves-for-image-classification\Plants_2\train'
test_directory = r'C:\Users\user\Desktop\4.1 study\plant-leaves-for-image-classification\Plants_2\test'
valid_directory = r'C:\Users\user\Desktop\4.1 study\plant-leaves-for-image-classification\Plants_2\valid'

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_dataframe,
    directory=train_directory,
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_dataframe,
    directory=test_directory,
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = test_datagen.flow_from_dataframe(
    dataframe=valid_dataframe,
    directory=valid_directory,
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4274 validated image filenames belonging to 2 classes.
Found 110 validated image filenames belonging to 2 classes.
Found 110 validated image filenames belonging to 2 classes.


In [10]:
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


9406464/9406464 [==============================] - 9s 1us/step


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=epochs, validation_data=valid_generator)


Epoch 1/10


134/134 [==============================] - 654s 5s/step - loss: 0.3953 - accuracy: 0.8234 - val_loss: 0.3863 - val_accuracy: 0.8182
Epoch 2/10
134/134 [==============================] - 557s 4s/step - loss: 0.2587 - accuracy: 0.8898 - val_loss: 0.3152 - val_accuracy: 0.8545
Epoch 3/10
134/134 [==============================] - 539s 4s/step - loss: 0.2345 - accuracy: 0.8949 - val_loss: 0.3597 - val_accuracy: 0.8364
Epoch 4/10
134/134 [==============================] - 521s 4s/step - loss: 0.2217 - accuracy: 0.9085 - val_loss: 0.3296 - val_accuracy: 0.8545
Epoch 5/10
134/134 [==============================] - 512s 4s/step - loss: 0.1940 - accuracy: 0.9212 - val_loss: 0.3658 - val_accuracy: 0.8455
Epoch 6/10
134/134 [==============================] - 541s 4s/step - loss: 0.1907 - accuracy: 0.9181 - val_loss: 0.4116 - val_accuracy: 0.8455
Epoch 7/10
134/134 [==============================] - 511s 4s/step - loss: 0.1930 - accuracy: 0.9263 - val_loss: 0.5602 - val_accuracy: 0.8

In [13]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

4/4 [==============================] - 12s 3s/step - loss: 0.3218 - accuracy: 0.8636
Test Loss: 0.3217659592628479
Test Accuracy: 0.8636363744735718
